 # 🦜 Flock - a declarative agent framework for Python



 [![GitHub](https://img.shields.io/github/stars/whiteducksoftware/flock?style=social)](https://github.com/whiteducksoftware/flock)



 Welcome to **Flock**, a powerful declarative agent framework for Python that simplifies agent creation and orchestration!



 ## 🎯 **What is Flock?**



 Flock is a framework focused on **declarative agent design** - instead of writing complex prompts, you simply:

 1. Declare what inputs an agent needs

 2. Specify what outputs you expect

 3. Let Flock handle the rest!



 ## 📚 **Documentation**



 For more information, visit:

 - https://github.com/whiteducksoftware/flock

 - https://whiteducksoftware.github.io/flock/



 ## 🧩 **What Will You Learn?**



 In this notebook, you'll learn how to:

 - Create simple agents with declarative inputs and outputs

 - Work with typed outputs and Python type hints

 - Integrate tools and code evaluation capabilities

 - Orchestrate multi-agent workflows with handoffs

 - Build a complete research assistant system with specialized agents



 Let's get started! 🚀

 ## 📦 Import the Needed Packages



 We'll start by importing all the necessary libraries for our examples:

 - Core Flock components for building and managing agents

 - Themes for enhancing output visualization

 - Basic tools for web search, coding, and other capabilities

 - Routers for agent orchestration and handoffs

 - Standard Python libraries for data processing

In [ ]:
from flock.core import Flock, FlockFactory
from flock.core.logging.formatters.themes import OutputTheme
from flock.core.tools import basic_tools
from flock.routers.agent.agent_router import AgentRouter, AgentRouterConfig
from typing import Literal, List, Dict
import os
import json
import re
import random
from datetime import datetime


 ## 🔰 Example 01: Simple Agent



 This first example demonstrates the core concept behind Flock - **declarative agent design**.



 ### 📝 **What You'll Learn**



 - How to create a basic Flock instance

 - How to define an agent with clear inputs and outputs

 - How to run an agent with specified input



 ### 💡 **Key Concepts**



 With Flock, you don't need to write complex prompts. Instead, you simply:

 1. Declare what the agent receives as input (`blog_idea`)

 2. Specify what the agent produces as output (`funny_blog_title, blog_headers`)

 3. Let Flock handle the rest!



 The framework determines how to achieve the transformation without you having to specify implementation details.

In [ ]:
MODEL = "azure/ara-gpt4o"
os.environ["AZURE_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"]
os.environ["AZURE_API_KEY"] = os.environ["AZURE_OPENAI_API_KEY"]
os.environ["AZURE_API_BASE"] = os.environ["AZURE_OPENAI_ENDPOINT"]

flock = Flock(model=MODEL)

bloggy = FlockFactory.create_default_agent(
    name="bloggy",
    input="blog_idea",
    output="funny_blog_title, blog_headers",
)
flock.add_agent(bloggy)

await flock.run_async(
    start_agent=bloggy, 
    input={"blog_idea": "A blog about cats"}
)


 ## 📊 Example 02: Typed Output



 This example takes things further by showing how to work with **typed outputs** in Flock.



 ### 📝 **What You'll Learn**



 - How to define structured outputs using Python type hints

 - How to incorporate tools for web content processing

 - How to use rich tables for beautiful output display

 - How to access the resulting Python objects with proper types



 ### 💡 **Key Concepts**



 The power of Flock becomes more apparent when you need structured data:

 - Use standard Python type hints: `list[str]`, `dict[str, str]`, `Literal[...]`

 - Add tools to extend agent capabilities (like web scraping)

 - Enable rich tables with custom themes for visually appealing output

 - Access the results as properly typed Python objects

In [ ]:
web_analyzer = FlockFactory.create_default_agent(
    name="web_analyzer",
    input="url",
    output="title, headings: list[str], "
            "entities_and_metadata: list[dict[str, str]], "
            "type: Literal['news', 'blog', 'opinion piece', 'tweet']",
    tools=[basic_tools.get_web_content_as_markdown],
    enable_rich_tables=True,
    output_theme=OutputTheme.aardvark_blue,
)
flock.add_agent(web_analyzer)

result = await flock.run_async(
    start_agent=web_analyzer,
    input={"url": "https://lite.cnn.com/travel/alexander-the-great-macedon-persian-empire-darius/index.html"},
)

# Print the title from the result (result is a Python object with proper types)
print(f"Title: {result.title}")
print(f"Content type: {result.type}")
print(f"First few headings: {result.headings[:3]}")


 ## 🛠️ Example 03: Tools and Code Execution



 This example demonstrates how to integrate **external tools** and **code execution** in Flock agents.



 ### 📝 **What You'll Learn**



 - How to provide agents with tool capabilities

 - How to enable code execution (Python evaluation)

 - How to use caching for improved efficiency

 - How to customize the visual presentation with themes



 ### 💡 **Key Concepts**



 Flock agents become more powerful when they can:

 - Search the web to gather information (`web_search_duckduckgo`)

 - Write and execute Python code to process data (`code_eval`)

 - Cache results for repeated calls with the same input

 - Present results with visually appealing themes

In [ ]:
celebrity_age_agent = FlockFactory.create_default_agent(
    name="celebrity_age_agent",
    input="a_person",
    output="persons_age_in_days",
    tools=[basic_tools.web_search_duckduckgo, basic_tools.code_eval],
    enable_rich_tables=True,
    output_theme=OutputTheme.adventuretime,
    use_cache=True,
)
flock.add_agent(celebrity_age_agent)

result = await flock.run_asyncn(
    start_agent=celebrity_age_agent,
    input={"a_person": "Johnny Depp"},
)


 ## 🔄 Example 04: Agent Handoff



 This example shows how to create **multi-agent workflows** with Flock using routers for orchestration.



 ### 📝 **What You'll Learn**



 - How to create multiple specialized agents

 - How to configure agent handoffs with routers

 - How to build sequential workflows

 - How to enable interactive mode with wait_for_input



 ### 💡 **Key Concepts**



 Flock shines when creating agent workflows:

 - Each agent specializes in a specific part of the task

 - Routers determine when and how to pass control between agents

 - AgentRouter with `with_output=True` passes agent outputs to the next agent

 - The `wait_for_input` parameter allows for human review between steps

In [ ]:
idea_agent = FlockFactory.create_default_agent(
    name="idea_agent",
    input="query",
    output="a_fun_software_project_idea",
    enable_rich_tables=True,
    wait_for_input=True,
)

project_plan_agent = FlockFactory.create_default_agent(
    name="project_plan_agent",
    input="a_fun_software_project_idea",
    output="catchy_project_name, project_pitch, techstack, project_implementation_plan",
    enable_rich_tables=True,
    wait_for_input=True,
)

# Set up AgentRouter for idea_agent to hand off to project_plan_agent
idea_agent.handoff_router = AgentRouter(config=AgentRouterConfig(with_output=True))

# Add agents to flock
flock.add_agent(idea_agent)
flock.add_agent(project_plan_agent)

# Run the flock with the idea agent as the starting point
result = await flock.run_async(
    input={"query": "fun software project idea about artificial intelligence"},
    start_agent=idea_agent,
)


 ## 🔬 Example 05: Research Assistant



 This comprehensive example rebuilds the research assistant from the semantic kernel notebook using Flock.

 It demonstrates the full power of Flock for creating complex, multi-agent systems.



 ### 📝 **What You'll Learn**



 - How to build a complete research workflow with multiple specialized agents

 - How to create custom tools as Python functions

 - How to structure agent interactions for complex tasks

 - How to process and generate different types of research content



 ### 💡 **Key Components**



 The research assistant system consists of:



 1. **ArXiv Search Agent** - Searches academic papers on arXiv

 2. **Web Search Agent** - Performs general web searches

 3. **Summary Agent** - Synthesizes information from multiple sources

 4. **Visualization Agent** - Creates data visualizations

 5. **Research Coordinator** - Orchestrates the entire research process



 Each agent has been assigned a unique theme for visual distinction, and the coordinator uses a router to manage the workflow.

In [ ]:
# Create plugins as tools

def search_arxiv(query: str, max_results: int = 5) -> str:
    """Search for academic papers on arXiv based on a query."""
    try:
        # Mock implementation (in a real application, you'd use the arxiv library)
        papers = [
            {
                "title": f"Understanding {query}: A Comprehensive Survey",
                "authors": "A. Researcher, B. Scientist",
                "summary": f"This paper provides a comprehensive overview of {query}...",
                "published": "2023-05-15",
                "url": f"https://arxiv.org/abs/2305.12345",
                "arxiv_id": "2305.12345",
                "local_path": f"papers/{query.replace(' ', '_')}_survey.pdf",
                "download_status": "success",
            },
            {
                "title": f"Recent Advances in {query} Research",
                "authors": "C. Expert, D. Scholar",
                "summary": f"This paper examines the latest developments in {query}...",
                "published": "2023-06-20",
                "url": f"https://arxiv.org/abs/2306.54321",
                "arxiv_id": "2306.54321",
                "local_path": f"papers/{query.replace(' ', '_')}_advances.pdf",
                "download_status": "success",
            },
        ]
        return json.dumps({"status": "success", "papers": papers[:max_results]}, indent=2)
    except Exception as e:
        return json.dumps({"status": "error", "message": str(e)})

def search_web(query: str, max_results: int = 5) -> str:
    """Perform a web search using DuckDuckGo for a given query."""
    try:
        # Mock implementation (in a real application, you'd use DuckDuckGo API)
        results = [
            {
                "title": f"{query} - Wikipedia",
                "href": f"https://en.wikipedia.org/wiki/{query.replace(' ', '_')}",
                "body": f"This article is about {query}. {query} is...",
            },
            {
                "title": f"The Ultimate Guide to {query}",
                "href": f"https://example.com/guide-to-{query.replace(' ', '-')}",
                "body": f"Learn everything you need to know about {query}...",
            },
        ]
        return json.dumps({"status": "success", "results": results[:max_results]}, indent=2)
    except Exception as e:
        return json.dumps({"status": "error", "message": str(e)})

def generate_visualization_data(topic: str) -> str:
    """Generate sample data for visualization based on a research topic."""
    # Extract words from the topic for category names
    topic_words = re.findall(r"\w+", topic.lower())
    
    # Generate category names based on topic words
    categories = []
    for i in range(5):  # Generate 5 categories
        if topic_words:
            word = random.choice(topic_words)
            categories.append(f"{word.capitalize()} {i+1}")
        else:
            categories.append(f"Category {i+1}")
    
    # Generate random metrics for visualization
    impact_factor = [random.randint(10, 100) for _ in range(len(categories))]
    research_activity = [random.randint(20, 80) for _ in range(len(categories))]
    
    # Create the data structure for visualization
    data = {
        "categories": categories,
        "metrics": {
            "Impact Factor": impact_factor,
            "Research Activity": research_activity,
        },
    }
    
    return json.dumps(data, indent=2)

def create_visualization(data_json: str, chart_type: str = "bar", topic: str = "Research Topic") -> str:
    """Mock function to create a visualization from the provided data."""
    # In a real implementation, this would create actual charts using matplotlib
    chart_filename = f"{chart_type}_{topic.replace(' ', '_')}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.png"
    return f"Visualization created and saved to visualizations/{chart_filename}"


 ### 🧩 **Research Agent Architecture**



 Below we create the specialized agents that make up our research system:



 - Each agent has a specific role in the research process

 - Each receives specific inputs and produces defined outputs

 - Each has access to the tools it needs for its task

 - Each gets a unique theme for visual distinction



 The coordinator uses an AgentRouter to determine the workflow, deciding which

 specialized agent to call at each step of the research process.

In [ ]:
# Create research flock
research_flock = Flock(model=MODEL)

# Create agents for the research assistant

# 1. ArXiv Search Agent
arxiv_agent = FlockFactory.create_default_agent(
    name="arxiv_agent",
    input="query: str, max_results: int = 5",
    output="arxiv_results: str",
    tools=[search_arxiv],
    enable_rich_tables=True,
    output_theme=OutputTheme.monokai,
)

# 2. Web Search Agent
web_agent = FlockFactory.create_default_agent(
    name="web_agent",
    input="query: str, max_results: int = 5",
    output="web_results: str",
    tools=[search_web, basic_tools.get_web_content_as_markdown],
    enable_rich_tables=True,
    output_theme=OutputTheme.dracula,
)

# 3. Summary Agent
summary_agent = FlockFactory.create_default_agent(
    name="summary_agent",
    input="topic: str, arxiv_results: str, web_results: str",
    output="research_summary: str",
    enable_rich_tables=True,
    output_theme=OutputTheme.one_dark,
)

# 4. Visualization Agent
visualization_agent = FlockFactory.create_default_agent(
    name="visualization_agent",
    input="topic: str",
    output="visualization_data: str, chart_type: str, visualization_path: str",
    tools=[generate_visualization_data, create_visualization],
    enable_rich_tables=True,
    output_theme=OutputTheme.synthwave,
)

# 5. Research Coordinator Agent
research_coordinator = FlockFactory.create_default_agent(
    name="research_coordinator",
    input="research_topic: str",
    output="final_research_report: str",
    enable_rich_tables=True,
    output_theme=OutputTheme.github_dark,
)


 ### 🔄 **Orchestrating the Research Workflow**



 Now we'll set up the coordinator with a router and add all agents to the flock.

 The research coordinator will:



 1. Receive the research topic from the user

 2. Decide which specialized agents to call and in what order

 3. Integrate their outputs into a cohesive research report

 4. Return the final report to the user



 This showcases how Flock can create complex, multi-agent workflows with minimal code.

In [ ]:
from flock.routers.default.default_router import DefaultRouter,DefaultRouterConfig
# Add router for the coordinator to orchestrate the research workflow
research_coordinator.handoff_router = AgentRouter(config=AgentRouterConfig(with_output=True))

arxiv_agent.handoff_router = DefaultRouter(config=DefaultRouterConfig(hand_off=research_coordinator.name))
web_agent.handoff_router = DefaultRouter(config=DefaultRouterConfig(hand_off=research_coordinator.name))
summary_agent.handoff_router = DefaultRouter(config=DefaultRouterConfig(hand_off=research_coordinator.name))
visualization_agent.handoff_router = DefaultRouter(config=DefaultRouterConfig(hand_off=research_coordinator.name))
research_coordinator.handoff_router = DefaultRouter(config=DefaultRouterConfig(hand_off=research_coordinator.name))


# Add all agents to the flock
research_flock.add_agent(arxiv_agent)
research_flock.add_agent(web_agent)
research_flock.add_agent(summary_agent)
research_flock.add_agent(visualization_agent)
research_flock.add_agent(research_coordinator)

# Run the research assistant
research_result = await research_flock.run_async(
    start_agent=research_coordinator,
    input={"research_topic": "quantum computing applications in healthcare"},
)

# Print a snippet of the final report
print("Research completed! Here's a preview of the report:")
if hasattr(research_result, 'final_research_report'):
    print(research_result.final_research_report[:500] + "...")


 ## 🎓 **Conclusion**



 In this notebook, you've learned how to use Flock to create:



 - Simple, declarative agents without complex prompts

 - Agents with typed outputs for structured data

 - Tool-using agents that can search the web and execute code

 - Multi-agent workflows with automatic handoffs

 - A complete research assistant system with specialized agents



 ### 📚 **Next Steps**



 To continue your journey with Flock:



 1. Explore the [official documentation](https://whiteducksoftware.github.io/flock/)

 2. Try creating your own specialized agents

 3. Experiment with different router configurations

 4. Implement custom tools for your specific use cases

 5. Build multi-agent systems for your own applications



 Flock makes it easy to build powerful AI agent systems with minimal code - perfect for

 both prototyping and production applications.